# Training notebook for some audio classification

From https://github.com/walkwithfastai/walkwithfastai.github.io/blob/master/nbs/course2020/vision/07_Audio.ipynb

### Docs

https://fastaudio.github.io/

https://github.com/musikalkemist/pytorchforaudio

* Use some Sound Event Detection - polyphonic
* Use image transformers - see guy at Kaggle bird competition
* Use multiple networks at different time scales and combine results
* Recurrent/transformer?
* change background noises
* use stereo signals to help - phase info
* Time of Day?
* Use wavelets

* Change validation so that it makes a class decision based on the whole validation sample, not just the 5 sec clip.

https://pywavelets.readthedocs.io/en/latest/index.html

https://pytorch-wavelets.readthedocs.io/en/latest/readme.html
https://www.researchgate.net/publication/242415506_BIRD_SOUND_CLASSIFICATION_AND_RECOGNITION_USING_WAVELETS_KLASIFIKACIJA_IN_PREPOZNAVANJE_PTIOEJIH_GLASOV_S_POMOOEJO_VALOEKOV

https://ataspinar.com/2018/12/21/a-guide-for-using-the-wavelet-transform-in-machine-learning/

https://github.com/tomrunia/PyTorchWavelets  - 1D wavelets in pytorch


* Related problem - human activity recognition https://machinelearningmastery.com/deep-learning-models-for-human-activity-recognition/




In [1]:
#torch.multiprocessing.set_start_method('spawn')

## Project setup

In [2]:
from pathlib import Path
import os

project_base = "/media/SSD/Sound Detection/"
results_folder = "Results"
data_folder = "birdclef-2021/train_short_audio"
model_folder = "models"

model_name_base = "spec-xresnet34"

In [3]:
data_path = Path(project_base)/data_folder
print(data_path)

/media/SSD/Sound Detection/birdclef-2021/train_short_audio


### Experiment documentation

In [4]:
%%javascript
var nb = IPython.notebook;
var kernel = IPython.notebook.kernel;
var command = "NOTEBOOK_FULL_PATH = '" + nb.base_url + nb.notebook_path + "'";
kernel.execute(command);



<IPython.core.display.Javascript object>

In [5]:
print("NOTEBOOK_FULL_PATH:\n", NOTEBOOK_FULL_PATH)

NOTEBOOK_FULL_PATH:
 /Sound/Training-GPU Spectrum-xresnet34-v2.ipynb


In [6]:
nb_path = Path(NOTEBOOK_FULL_PATH)
print(nb_path)

# Make result folder if it does not exist
result_path = Path(project_base, results_folder, nb_path.stem)
print(result_path)
os.makedirs(result_path, exist_ok=True)

/Sound/Training-GPU Spectrum-xresnet34-v2.ipynb
/media/SSD/Sound Detection/Results/Training-GPU Spectrum-xresnet34-v2


In [7]:
model_out_path = Path(project_base) / model_folder
print(model_out_path)
os.makedirs(model_out_path, exist_ok=True)

/media/SSD/Sound Detection/models


## Start here

In [8]:
import torch
import pandas as pd
import torchaudio
import numpy as np

Ideas
* Convert dataframe into sqlite in memory database for faster training
* Convert dataframe to numpy array for faster training


In [9]:
# from https://pytorch.org/tutorials/beginner/audio_preprocessing_tutorial.html#preparing-data-and-utility-functions-skip-this-section

from IPython.display import Audio, display
import librosa
import matplotlib.pyplot as plt


def play_audio(waveform, sample_rate):
    if type(waveform) != np.ndarray:
        waveform = waveform.numpy()
    num_channels, num_frames = waveform.shape
    if num_channels == 1:
        display(Audio(waveform[0], rate=sample_rate))
    elif num_channels == 2:
        display(Audio((waveform[0], waveform[1]), rate=sample_rate))
    else:
        raise ValueError(
            "Waveform with more than 2 channels are not supported.")


def print_stats(waveform, sample_rate=None, src=None):
    if src:
        print("-" * 10)
        print("Source:", src)
        print("-" * 10)
    if sample_rate:
        print("Sample Rate:", sample_rate)
        print("Shape:", tuple(waveform.shape))
        print("Dtype:", waveform.dtype)
        print(f" - Max:     {waveform.max().item():6.3f}")
        print(f" - Min:     {waveform.min().item():6.3f}")
        print(f" - Mean:    {waveform.mean().item():6.3f}")
        print(f" - Std Dev: {waveform.std().item():6.3f}")
        print()
        print(waveform)
        print()


def plot_waveform(waveform,
                  sample_rate,
                  title="Waveform",
                  xlim=None,
                  ylim=None):
    if type(waveform) != np.ndarray:
        waveform = waveform.numpy()
    num_channels, num_frames = waveform.shape
    time_axis = torch.arange(0, num_frames) / sample_rate
    figure, axes = plt.subplots(num_channels, figsize=(12, 6))
    if num_channels == 1:
        axes = [axes]
    for c in range(num_channels):
        axes[c].plot(time_axis, waveform[c], linewidth=1, color="#A300F9")
        axes[c].grid(True)
        if num_channels > 1:
            axes[c].set_ylabel(f'Channel {c+1}')
        if xlim:
            axes[c].set_xlim(xlim)
        if ylim:
            axes[c].set_ylim(ylim)
    figure.suptitle(title)
    plt.show(block=False)


def plot_specgram(waveform, sample_rate, title="Spectrogram", xlim=None):
    waveform = waveform.numpy()

    num_channels, num_frames = waveform.shape
    time_axis = torch.arange(0, num_frames) / sample_rate

    figure, axes = plt.subplots(num_channels, 1, figsize=(12, 6))
    if num_channels == 1:
        axes = [axes]
    for c in range(num_channels):
        axes[c].specgram(waveform[c], Fs=sample_rate)
        if num_channels > 1:
            axes[c].set_ylabel(f'Channel {c+1}')
        if xlim:
            axes[c].set_xlim(xlim)
    figure.suptitle(title)
    plt.show(block=False)


def plot_spectrogram(spec,
                     title=None,
                     ylabel='freq_bin',
                     aspect='auto',
                     xmax=None):
    fig, axs = plt.subplots(1, 1)
    axs.set_title(title or 'Spectrogram (db)')
    axs.set_ylabel(ylabel)
    axs.set_xlabel('frame')
    im = axs.imshow(librosa.power_to_db(spec), origin='lower', aspect=aspect)
    if xmax:
        axs.set_xlim((0, xmax))
    fig.colorbar(im, ax=axs)
    plt.show(block=False)

In [10]:
df = pd.read_csv('/media/SSD/Sound Detection/birdclef-2021/enhanced_train_metadata.csv')

## Handle class labels

In [11]:
primary_labels = set(df.primary_label.unique())

In [12]:
BIRD_CODE = {}
for i, label in enumerate(sorted(primary_labels)):
    BIRD_CODE[label] = i

INV_BIRD_CODE = np.array(sorted(primary_labels))

In [13]:
INV_BIRD_CODE

array(['acafly', 'acowoo', 'aldfly', 'ameavo', 'amecro', 'amegfi',
       'amekes', 'amepip', 'amered', 'amerob', 'amewig', 'amtspa',
       'andsol1', 'annhum', 'astfly', 'azaspi1', 'babwar', 'baleag',
       'balori', 'banana', 'banswa', 'banwre1', 'barant1', 'barswa',
       'batpig1', 'bawswa1', 'bawwar', 'baywre1', 'bbwduc', 'bcnher',
       'belkin1', 'belvir', 'bewwre', 'bkbmag1', 'bkbplo', 'bkbwar',
       'bkcchi', 'bkhgro', 'bkmtou1', 'bknsti', 'blbgra1', 'blbthr1',
       'blcjay1', 'blctan1', 'blhpar1', 'blkpho', 'blsspa1', 'blugrb1',
       'blujay', 'bncfly', 'bnhcow', 'bobfly1', 'bongul', 'botgra',
       'brbmot1', 'brbsol1', 'brcvir1', 'brebla', 'brncre', 'brnjay',
       'brnthr', 'brratt1', 'brwhaw', 'brwpar1', 'btbwar', 'btnwar',
       'btywar', 'bucmot2', 'buggna', 'bugtan', 'buhvir', 'bulori',
       'burwar1', 'bushti', 'butsal1', 'buwtea', 'cacgoo1', 'cacwre',
       'calqua', 'caltow', 'cangoo', 'canwar', 'carchi', 'carwre',
       'casfin', 'caskin', 'caster1

## Create a dataset

https://debuggercafe.com/multi-label-image-classification-with-pytorch-and-deep-learning/

https://github.com/gskdhiman/Pytorch-Transfer-learning-Multi-Label-image-Classification/blob/master/pytorch-resnet50-starter.ipynb




In [14]:
class Dataset_from_df(torch.utils.data.Dataset):
    def __init__(self, df, path, crop_length):
        self.df = df
        self.path = path
        self.crop_length = crop_length
        self.text_labels = set(df.primary_label.unique())
        self.class_n = len(self.text_labels)
        self.class_code = {}
        for i, label in enumerate(sorted(self.text_labels)):
            self.class_code[label] = i

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        def _randint(a, b):
            return torch.randint(a, b, (1, )).item()

        #meta = torchaudio.info(self.path + "/" + df.iloc[index]['subpath'])
        label = self.df.iloc[index]['primary_label']
        length = self.df.iloc[index]['sample_len']
        c = self.df.iloc[index]['channels']

        #need to map sample label to index

        if length == self.crop_length:
            x, sr = torchaudio.load(self.path + "/" +
                                    self.df.iloc[index]['subpath'])
        elif length > self.crop_length:
            crop_start = _randint(0, int(length - self.crop_length))
            x, sr = torchaudio.load(self.path + "/" +
                                    self.df.iloc[index]['subpath'],
                                    frame_offset=crop_start,
                                    num_frames=self.crop_length)
        else:
            s, sr = torchaudio.load(self.path + "/" +
                                    self.df.iloc[index]['subpath'])

            zeros_front = (_randint(0, self.crop_length - length))
            pad_front = torch.zeros((c, zeros_front))
            pad_back = torch.zeros(
                (c, self.crop_length - length - zeros_front))
            waveform = torch.cat((pad_front, s, pad_back), 1)
            x = waveform

        target = torch.zeros(self.class_n)
        target[self.class_code[label]] = 1.0
        #print("label=",label, "code=",self.class_code[label])

        return x, sr, target

## Split the dataset

In [15]:
dataset = Dataset_from_df(df,'/media/SSD/Sound Detection/birdclef-2021/train_short_audio',5*32000)

In [16]:
valid_n = int(len(dataset) * 0.20)
training_data, validation_data = torch.utils.data.random_split(
    dataset, [len(dataset) - valid_n, valid_n],
    generator=torch.Generator().manual_seed(42))

print(
    f"Training Data len {len(training_data)} Validation Data len {len(validation_data)}"
)

Training Data len 50300 Validation Data len 12574


In [17]:
x,sr,y = training_data[0]

In [18]:
x.shape

torch.Size([1, 160000])

In [ ]:
torch.max(y)

In [ ]:
play_audio(waveform=x, sample_rate=sr)

In [ ]:
plot_waveform(waveform=x, sample_rate=sr, title="Waveform", xlim=None, ylim=None)

In [ ]:
plot_specgram(waveform=x, sample_rate=sr, title="Spectrogram", xlim=None)

In [ ]:
#debug
#training_data=torch.utils.data.Subset(training_data, range(0,1024))
#validation_data=torch.utils.data.Subset(validation_data, range(0,1024))
#print(f"Training Data len {len(training_data)} Validation Data len {len(validation_data)}")

In [19]:
train_dataloader = torch.utils.data.DataLoader(dataset=training_data,
                                               batch_size=16,
                                               num_workers=12,
                                               pin_memory=True,
                                               shuffle=True)

In [20]:
validation_dataloader = torch.utils.data.DataLoader(dataset=validation_data,
                                                    batch_size=16,
                                                    num_workers=12,
                                                    pin_memory=True,
                                                    shuffle=False)

In [19]:
from fastai.vision.all import *

In [20]:
# build a new model
model = xresnet34(pretrained=True)
layer = model[0][0]
layer.in_channels = 1
layer.weight = nn.Parameter(layer.weight[:, 1, :, :].unsqueeze(1))
model[0][0] = layer

resample = torchaudio.transforms.Resample(32000, 48000)

spec = torchaudio.transforms.MelSpectrogram(sample_rate=48000,
                                            n_fft=768,
                                            n_mels=80,
                                            f_min=200,
                                            f_max=10000)
a_to_db = torchaudio.transforms.AmplitudeToDB()
#new_model = nn.Sequential(resample,spec,model)
new_model = nn.Sequential(resample, spec, a_to_db, model)

In [21]:
#for param in new_model.parameters():
#    param.requires_grad = False

In [22]:
# Change the output layer to match class n
#new_model[-1][-1]=nn.Linear(in_features=512,out_features=len(primary_labels),bias=True)
new_model[-1][-1]=nn.Linear(in_features=512,out_features=len(primary_labels))

In [23]:
new_model.cuda()

Sequential(
  (0): Resample()
  (1): MelSpectrogram(
    (spectrogram): Spectrogram()
    (mel_scale): MelScale()
  )
  (2): AmplitudeToDB()
  (3): XResNet(
    (0): ConvLayer(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): ConvLayer(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (2): ConvLayer(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): ResBlock(
        (convpath): Sequential(
          (0):

In [25]:
model_out_str = str(model_out_path/(model_name_base+"2.pth"))
new_model.load_state_dict(torch.load(model_out_str))

<All keys matched successfully>

In [27]:
new_model[-1][-2]=nn.Dropout(p=0.5,inplace=False)

In [28]:
new_model

Sequential(
  (0): Resample()
  (1): MelSpectrogram(
    (spectrogram): Spectrogram()
    (mel_scale): MelScale()
  )
  (2): AmplitudeToDB()
  (3): XResNet(
    (0): ConvLayer(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): ConvLayer(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (2): ConvLayer(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): ResBlock(
        (convpath): Sequential(
          (0):

## Training functions

In [29]:
#def n_fn(*args): return 0

In [30]:
def trainfp32(epoch, train_dataloader, optimizer, loss_fn, lr_scheduler,
              metric_fn, model):

    #lr = adjust_learning_rate(optimizer, epoch-epoch_start)

    #for param_group in optimizer.param_groups:
    #   param_group["lr"] = lr

    #print("Epoch = {}, lr = {}".format(epoch, optimizer.param_groups[0]["lr"]))

    model.train()

    training_loss = 0
    metric = 0

    pbar = tqdm(total=len(train_dataloader))

    for batch in train_dataloader:

        optimizer.zero_grad()

        x, sr, y = batch

        x = x.cuda(non_blocking=True)
        y = y.cuda(non_blocking=True)

        output = model(x)

        loss = loss_fn(output, y)
        #print("outputmax =",torch.max(output, 1))
        #print("ymax=",torch.max(y, 1))

        preds = torch.sigmoid(output).data > 0.5
        preds = preds.to(torch.float32)

        loss.backward()

        optimizer.step()

        lr_scheduler.step()

        training_loss += loss.data.item() * x.size(0)

        #np_target = target.to("cpu").to(torch.int).numpy()
        #np_preds = preds.to("cpu").to(torch.int).numpy()

        #metric += f1_score(np_target ,np_preds , average="samples")  * data.size(0)

        metric += metric_fn(y, preds) * x.size(0)

        pbar.update(1)
    pbar.close()
    training_loss /= len(train_dataloader.dataset)
    metric /= len(train_dataloader.dataset)
    return training_loss, metric

In [31]:
def validatefp32(valid_dataloader, loss_fn, metric_fn, model):
    validation_loss = 0
    metric = 0
    model.eval()
    pbar = tqdm(total=len(valid_dataloader))
    for batch in valid_dataloader:
        x, sr, y = batch
        x = x.cuda(non_blocking=True)
        y = y.cuda(non_blocking=True)
        output = model(x)
        loss = loss_fn(output, y)
        preds = torch.sigmoid(output).data > 0.5
        preds = preds.to(torch.float32)
        validation_loss += loss.data.item() * x.size(0)
        metric += metric_fn(y, preds) * x.size(0)
        pbar.update(1)
    pbar.close()
    validation_loss /= len(valid_dataloader.dataset)
    metric /= len(valid_dataloader.dataset)
    return validation_loss, metric

In [32]:
def predictfp32(batch, model):
    x, sr, y = batch
    x = x.cuda(non_blocking=True)
    y = y.cuda(non_blocking=True)
    output = model(x)
    preds = torch.sigmoid(output).data > 0.5
    preds = preds.to(torch.float32)
    return output, preds

In [33]:
init_lr = .001
epochs=20*6

In [34]:
optimizer = torch.optim.Adam(new_model.parameters(), lr=init_lr, weight_decay=0)

In [35]:
loss_fn = torch.nn.BCEWithLogitsLoss()

In [36]:
lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=.0001, steps_per_epoch=len(train_dataloader),
                                                  epochs=epochs)

In [37]:
from sklearn.metrics import f1_score, accuracy_score


def f1_torch(preds, target):
    np_target = target.to("cpu").to(torch.int).numpy()
    np_preds = preds.to("cpu").to(torch.int).numpy()
    return f1_score(np_target, np_preds, average="micro", zero_division=0)


def accuracy_torch(preds, target):
    np_target = target.to("cpu").to(torch.int).numpy()
    np_preds = preds.to("cpu").to(torch.int).numpy()
    return accuracy_score(np_target, np_preds)

In [38]:
from tqdm.notebook import tqdm

## Training Loop

In [39]:
for epoch in range(1, epochs + 1):

    lr = lr_scheduler.get_last_lr()
    print('Epoch: {}, LR: {:.3e}'.format(epoch, lr[0]), end="--")

    training_loss, training_metric = trainfp32(epoch,
                                               train_dataloader,
                                               optimizer,
                                               loss_fn,
                                               lr_scheduler,
                                               metric_fn=accuracy_torch,
                                               model=new_model)

    print('Training-- Loss: {:.4f}, Metric: {:.3f}'.format(
        training_loss, training_metric),
          end=',')
    validation_loss, validation_metric = validatefp32(validation_dataloader,
                                                      loss_fn=loss_fn,
                                                      metric_fn=accuracy_torch,
                                                      model=new_model)

    #save_checkpoint(NOTEBOOK_FULL_PATH, model=newmodel, epoch, valid_error)

    print('Validation-- Loss: {:.4f}, Metric = {:.3f}'.format(
        validation_loss, validation_metric))

Epoch: 1, LR: 4.000e-06--

  0%|          | 0/3144 [00:00<?, ?it/s]

Training-- Loss: 0.0039, Metric: 0.661,

  0%|          | 0/786 [00:00<?, ?it/s]

Validation-- Loss: 0.0060, Metric = 0.570
Epoch: 2, LR: 4.183e-06--

...

Validation-- Loss: 0.0071, Metric = 0.589
Epoch: 120, LR: 3.534e-08--

  0%|          | 0/3144 [00:00<?, ?it/s]

Training-- Loss: 0.0022, Metric: 0.804,

  0%|          | 0/786 [00:00<?, ?it/s]

Validation-- Loss: 0.0073, Metric = 0.589


In [ ]:
    validation_loss, validation_metric = validatefp32(validation_dataloader,
                                                      loss_fn=loss_fn,
                                                      metric_fn=accuracy_torch,
                                                      model=new_model)

In [ ]:
print('Validation-- Loss: {:.4f}, Metric = {:.3f}'.format(validation_loss, validation_metric))

## make a prediction

In [ ]:
new_model.eval();

In [21]:
batch = next(iter(validation_dataloader)) 
output, preds = predictfp32(batch,model=new_model)

NameError: name 'predictfp32' is not defined

In [23]:
batch[0].shape

torch.Size([16, 1, 160000])

In [ ]:
output.topk(3)

In [ ]:
batch[2].topk(1)

## Save pytorch model

In [40]:
model_out_str = str(model_out_path/(model_name_base+"3.pth"))
torch.save(new_model.state_dict(), model_out_str)
